In [2]:
import pandas as pd 
import numpy as np
import copy

#data = pd.read_csv("train.csv") 
#ran = np.random.rand(len(data)) < 0.8
#train = data[ran]
#val = data[~ran]
#train.to_csv('UsedTrainingData.csv')
#val.to_csv('UsedValidationData.csv')

train =  pd.read_csv('UsedTrainingData.csv')
val = pd.read_csv('UsedValidationData.csv')

targetAttribute = 'left'
Attributes = list(train.columns)
Attributes.remove(targetAttribute)
examples = list(train.index.values) 

NumericalAttributes = ['satisfaction_level', 'last_evaluation', 'number_project', 'average_montly_hours', 'time_spend_company']
CategoricalAttributes = ['Work_accident', 'promotion_last_5years', 'sales', 'salary']

def getImpurityOfExamples(examplesForCurNode,method):
    NumberOfPosEx = 0
    NumberOfNegEx = 0
    for j in examplesForCurNode:
        if  str(train.at[int(j),targetAttribute]) == str(1):
            NumberOfPosEx = NumberOfPosEx+1
        elif str(train.at[int(j),targetAttribute]) == str(0):
             NumberOfNegEx = NumberOfNegEx+1
    PropOfPos =  NumberOfPosEx/len(examplesForCurNode)
    PropOfNeg = NumberOfNegEx/len(examplesForCurNode)
    if method == 'Gini':
        return (2*PropOfPos*PropOfNeg)
    elif method == 'misClasifi':
        return min(PropOfPos,PropOfNeg)
    elif method == 'Entropy':
        if PropOfPos == 0:
            return (-1*PropOfNeg*(np.log(PropOfNeg)/np.log(2)))
        elif PropOfNeg == 0:
            return (-1*PropOfPos*(np.log(PropOfPos)/np.log(2)))
        else:
            return ((-1*PropOfPos*(np.log(PropOfPos)/np.log(2))) + (-1*PropOfNeg*(np.log(PropOfNeg)/np.log(2))))

def getMethodImpurity(Attribute,value,examplesForCurAttr,method): 
    fra1 = train.loc[examplesForCurAttr,:]
    LessThanExamples = []
    MoreThanExamples = []
    if Attribute in NumericalAttributes: 
        for j in examplesForCurAttr:
            if  float(str(fra1.at[int(j),Attribute])) <= float(str(value)):
                LessThanExamples.append(int(j))
            else:
                MoreThanExamples.append(int(j))
    else:
        for j in examplesForCurAttr:
            if  str(fra1.at[int(j),Attribute]) == str(value):
                LessThanExamples.append(int(j))
            else:
                MoreThanExamples.append(int(j))
        
    if len(LessThanExamples) <=0 or len(MoreThanExamples) <= 0:    
        return 999
    else:  
        return ((len(LessThanExamples)*getImpurityOfExamples(LessThanExamples,method))+(len(MoreThanExamples)*getImpurityOfExamples(MoreThanExamples,method)))/len(examplesForCurAttr)
    
class Node(object):
    def __init__(self,examples,decisionAttributeType,decisionAttribute,decisionAttributeValue,level):
        self.examples = examples
        self.decisionAttributeType = decisionAttributeType
        self.decisionAttribute = decisionAttribute        
        self.decisionAttributeValue = decisionAttributeValue
        self.level = level
        self.leftChild = None
        self.rightChild = None
        
    def __str__(self): 
        return "Decision attribute type is "+self.decisionAttributeType+"decisionAttribute is "+ self.decisionAttribute+" decisionAttributeValue is "+ str(self.decisionAttributeValue)+" of level "+str(self.level)+" number of examples of this node "+str(len(self.examples))
    
def BestOfAttributes(examplesForCurAttr,targetAttribute,method):
    fra1 = train.loc[examplesForCurAttr,CategoricalAttributes]
    dictForCategAttrUniqueValues = {}
    for i in CategoricalAttributes:
        dictForCategAttrUniqueValues[i] = list(fra1[i].sort_values().unique())
    dictForEntropyUniqueValueCateg = copy.deepcopy(dictForCategAttrUniqueValues)
    print("calculating Impurities please wait")
    for i in CategoricalAttributes:
        for j,val in enumerate(dictForCategAttrUniqueValues[i]):
            dictForEntropyUniqueValueCateg[i][j] = getMethodImpurity(i,dictForCategAttrUniqueValues[i][j],examplesForCurAttr,method)          
    EntropyValuesForCategoricalAttributes = {}
    for i in CategoricalAttributes:
        EntropyValuesForCategoricalAttributes[i]=min(dictForEntropyUniqueValueCateg[i])
    BestOfCategoricalAttributes = min(EntropyValuesForCategoricalAttributes, key=EntropyValuesForCategoricalAttributes.get)
    BestOfCategoricalAttributesValue = min(EntropyValuesForCategoricalAttributes.values())   
    
    fra1 = train.loc[examplesForCurAttr,NumericalAttributes]
    dictForNumAttrUniqueValues = {}
    for i in NumericalAttributes:
        dictForNumAttrUniqueValues[i] = list(fra1[i].sort_values().unique())
    dictForEntropyUniqueValueNum = copy.deepcopy(dictForNumAttrUniqueValues)
    
    print("calculating Impurities please wait")
    for i in NumericalAttributes:
        for j,val in enumerate(dictForNumAttrUniqueValues[i]):
            dictForEntropyUniqueValueNum[i][j] = getMethodImpurity(i,dictForNumAttrUniqueValues[i][j],examplesForCurAttr,method)          
    EntropyValuesForNumericalAttributes = {}
    for i in NumericalAttributes:
        EntropyValuesForNumericalAttributes[i]=min(dictForEntropyUniqueValueNum[i])
    BestOfNumericalAttributes = min(EntropyValuesForNumericalAttributes, key=EntropyValuesForNumericalAttributes.get)
    BestOfNumericalAttributesValue = min(EntropyValuesForNumericalAttributes.values())
    
    for index, item in enumerate(dictForEntropyUniqueValueNum[BestOfNumericalAttributes]):
        if item == BestOfNumericalAttributesValue:
            BestSplitValueNumerical = dictForNumAttrUniqueValues[BestOfNumericalAttributes][index]
            break
            
    for index, item in enumerate(dictForEntropyUniqueValueCateg[BestOfCategoricalAttributes]):
        if item == BestOfCategoricalAttributesValue:
            BestSplitValueCategorical = dictForCategAttrUniqueValues[BestOfCategoricalAttributes][index]
            break

    if BestOfNumericalAttributesValue < BestOfCategoricalAttributesValue:
        return BestOfNumericalAttributes,BestSplitValueNumerical
    else: 
        return BestOfCategoricalAttributes,BestSplitValueCategorical
    
def DecisionTreeBuilder(examplesForCurNode,targetAttribute,level,method):
    allExPos = True
    for i in examplesForCurNode:
        if str(train.at[int(i),targetAttribute]) == str(1):
            pass
        else:
            allExPos = False
            break
    if allExPos:        
        r = Node(examplesForCurNode,'leafNode','leafNode','Positive',level)
        print(r)
        print("Type1")
        return r
    allExNeg = True
    for j in examplesForCurNode:
        if str(train.at[j,targetAttribute]) == str(0):
            pass
        else:
            allExNeg = False
            break
    if allExNeg:
        r = Node(examplesForCurNode,'leafNode','leafNode','Negative',level)
        print(r)
        print("Type2")
        return r
    if getImpurityOfExamples(examplesForCurNode,method) < 0.001 or len(examplesForCurNode)<10:
        fra1 = train.loc[examplesForCurNode,[targetAttribute] ]
        targetAttributesList = fra1[targetAttribute].values
        unique1, counts1 = np.unique(targetAttributesList, return_counts=True)
        dictOfTaAttr = dict(zip(unique1, counts1))
        posCount = dictOfTaAttr[1]
        negCount = dictOfTaAttr[0]
        if (posCount/(posCount+negCount))>0.5:
            r = Node(examplesForCurNode,'leafNode','leafNode','Positive',level)
            #numberOfNodes  = numberOfNodes + 1
        else:
            r = Node(examplesForCurNode,'leafNode','leafNode','Negative',level)
            #numberOfNodes  = numberOfNodes + 1
        print(r)
        print("Type3")
        return r
    
    BestAttribute,BestAttributeValue = BestOfAttributes(examplesForCurNode,targetAttribute,method)
    
    if BestAttribute in NumericalAttributes:
        BestAttributeType = 'numerical'
    else:
        BestAttributeType = 'categorical'
    r = Node(examplesForCurNode,BestAttributeType,BestAttribute,BestAttributeValue,level)
    print(r)
    print("Type4")
    #numberOfNodes  = numberOfNodes + 1
    examplesForLeftChildOfCurrentNode = []
    examplesForRightChildOfCurrentNode = []        
    if BestAttributeType == 'categorical':        
        for iter1 in examplesForCurNode:
            if  str(train.at[int(iter1),str(BestAttribute)]) == str(BestAttributeValue):
                examplesForLeftChildOfCurrentNode.append(int(iter1))
            else:
                examplesForRightChildOfCurrentNode.append(int(iter1))
    else:
        for iter1 in examplesForCurNode:
            if  float(str(train.at[int(iter1),str(BestAttribute)])) <= float(str(BestAttributeValue)):
                examplesForLeftChildOfCurrentNode.append(int(iter1))
            else:
                examplesForRightChildOfCurrentNode.append(int(iter1))
    r.leftChild = DecisionTreeBuilder(examplesForLeftChildOfCurrentNode,targetAttribute,level+1,method)
    r.rightChild = DecisionTreeBuilder(examplesForRightChildOfCurrentNode,targetAttribute,level+1,method)
    
    return r 


In [3]:
examplesForRootNode = examples[:]
method = 'Gini'
RootNode = DecisionTreeBuilder(examplesForRootNode,targetAttribute,0,method)

calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is satisfaction_level decisionAttributeValue is 0.46 of level 0 number of examples of this node 8989
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is number_project decisionAttributeValue is 2 of level 1 number of examples of this node 2501
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is last_evaluation decisionAttributeValue is 0.57 of level 2 number of examples of this node 1042
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is last_evaluation decisionAttributeValue is 0.44 of level 3 number of examples of this node 970
Type4
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is

calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is satisfaction_level decisionAttributeValue is 0.11 of level 2 number of examples of this node 1459
Type4
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Positive of level 3 number of examples of this node 543
Type1
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is number_project decisionAttributeValue is 6 of level 3 number of examples of this node 916
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is average_montly_hours decisionAttributeValue is 287 of level 4 number of examples of this node 908
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is last_evaluation decisionAttributeValue is 0.98 of level 5 number of ex

Decision attribute type is categoricaldecisionAttribute is sales decisionAttributeValue is accounting of level 10 number of examples of this node 28
Type4
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Positive of level 11 number of examples of this node 1
Type1
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is categoricaldecisionAttribute is sales decisionAttributeValue is sales of level 11 number of examples of this node 27
Type4
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Negative of level 12 number of examples of this node 5
Type3
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Negative of level 12 number of examples of this node 22
Type2
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is satisfaction_level decisionAttributeValue

calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is number_project decisionAttributeValue is 4 of level 6 number of examples of this node 22
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is average_montly_hours decisionAttributeValue is 188 of level 7 number of examples of this node 10
Type4
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Negative of level 8 number of examples of this node 3
Type2
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Positive of level 8 number of examples of this node 7
Type3
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is categoricaldecisionAttribute is sales decisionAttributeValue is support of level 7 number of examples of this node 12
Type4
Decision attribute type i

calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is categoricaldecisionAttribute is sales decisionAttributeValue is sales of level 8 number of examples of this node 4110
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is average_montly_hours decisionAttributeValue is 267 of level 9 number of examples of this node 1093
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is satisfaction_level decisionAttributeValue is 0.57 of level 10 number of examples of this node 1031
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is satisfaction_level decisionAttributeValue is 0.53 of level 11 number of examples of this node 183
Type4
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeVa

calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is average_montly_hours decisionAttributeValue is 139 of level 10 number of examples of this node 2995
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is satisfaction_level decisionAttributeValue is 0.51 of level 11 number of examples of this node 206
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is average_montly_hours decisionAttributeValue is 137 of level 12 number of examples of this node 15
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is categoricaldecisionAttribute is sales decisionAttributeValue is technical of level 13 number of examples of this node 14
Type4
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Negative of level 14 numb

calculating Impurities please wait
Decision attribute type is categoricaldecisionAttribute is salary decisionAttributeValue is low of level 8 number of examples of this node 718
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is average_montly_hours decisionAttributeValue is 273 of level 9 number of examples of this node 338
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is number_project decisionAttributeValue is 4 of level 10 number of examples of this node 330
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is last_evaluation decisionAttributeValue is 0.99 of level 11 number of examples of this node 261
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is number

Type2
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is last_evaluation decisionAttributeValue is 0.95 of level 5 number of examples of this node 96
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is last_evaluation decisionAttributeValue is 0.69 of level 6 number of examples of this node 92
Type4
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Negative of level 7 number of examples of this node 49
Type2
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is last_evaluation decisionAttributeValue is 0.7 of level 7 number of examples of this node 43
Type4
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Positive of level 8 number of examples of this node 5
Type3
calculating Im

Decision attribute type is categoricaldecisionAttribute is sales decisionAttributeValue is support of level 7 number of examples of this node 287
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is number_project decisionAttributeValue is 4 of level 8 number of examples of this node 18
Type4
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Negative of level 9 number of examples of this node 11
Type2
Decision attribute type is leafNodedecisionAttribute is leafNode decisionAttributeValue is Negative of level 9 number of examples of this node 7
Type3
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is average_montly_hours decisionAttributeValue is 132 of level 8 number of examples of this node 269
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute typ

Decision attribute type is numericaldecisionAttribute is satisfaction_level decisionAttributeValue is 0.92 of level 8 number of examples of this node 503
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is average_montly_hours decisionAttributeValue is 276 of level 9 number of examples of this node 502
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is categoricaldecisionAttribute is Work_accident decisionAttributeValue is 0 of level 10 number of examples of this node 500
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is average_montly_hours decisionAttributeValue is 223 of level 11 number of examples of this node 470
Type4
calculating Impurities please wait
calculating Impurities please wait
Decision attribute type is numericaldecisionAttribute is last_evaluation decisionA

In [4]:
def RecursiveFunctionToPredict(i,r,TestSet):
    if r.decisionAttribute == 'leafNode':
        if r.decisionAttributeValue == 'Positive':
            return '1'
        else:
            return '0'
    else:
        if r.decisionAttributeType == 'categorical':            
            AttrValue = str(TestSet.at[int(i),str(r.decisionAttribute)])            
            if AttrValue == str(r.decisionAttributeValue):
                return RecursiveFunctionToPredict(i,r.leftChild,TestSet)
            else:
                return RecursiveFunctionToPredict(i,r.rightChild,TestSet)            
        elif r.decisionAttributeType == 'numerical':            
            AttrValue = float(str(TestSet.at[int(i),str(r.decisionAttribute)]))            
            if AttrValue <= float(str(r.decisionAttributeValue)):
                return RecursiveFunctionToPredict(i,r.leftChild,TestSet)
            else:
                return RecursiveFunctionToPredict(i,r.rightChild,TestSet)
def predict(RootNode,NameOfTestFile):
    GivenTestData = pd.read_csv(str(NameOfTestFile))
    GivenTestData['predict']= [0]*len(GivenTestData)
    for i in list(GivenTestData.index.values):        
        GivenTestData.at[i,'predict'] = RecursiveFunctionToPredict(i,RootNode,GivenTestData)
    return GivenTestData            

In [8]:
    GivenTestData =  train.copy()
    GivenTestData['predict']= [0]*len(GivenTestData)
    for i in list(GivenTestData.index.values):        
        GivenTestData.at[i,'predict'] = RecursiveFunctionToPredict(i,RootNode,GivenTestData)
    
    TN = 0
    FP = 0
    FN = 0
    TP = 0
    for i in list(GivenTestData.index.values):
        if str(GivenTestData.at[i,'left']) == str(1) and str(GivenTestData.at[i,'predict']) == str(1):
            TP = TP + 1
        elif str(GivenTestData.at[i,'left']) == str(1) and str(GivenTestData.at[i,'predict']) == str(0):
            FN = FN + 1
        elif str(GivenTestData.at[i,'left']) == str(0) and str(GivenTestData.at[i,'predict']) == str(1):
            FP = FP + 1
        elif str(GivenTestData.at[i,'left']) == str(0) and str(GivenTestData.at[i,'predict']) == str(0):
            TN = TN + 1
    print("Results On Training Data")
    print("Using Gini Coefficient as impurity measure")
    print("-------------------------------------------------")
    print("True Positive are    "+ str(TP))
    print("True Negatives are   "+ str(TN))
    print("False Positive are   "+ str(FP))
    print("False Negatives are  "+str(FN))
    
    Precision = (TP/(TP+FP))
    Recall = (TP/(TP+FN))
    F1_Score = 2*((1/Recall)+(1/Precision))
    accuracy = (TN+TP)/(TN+TP+FP+FN)
    print("Precsion is          "+str(Precision))
    print("Recall is            "+str(Recall))
    print("F1_Score is          "+str(F1_Score))
    print("Accuracy is          "+str(accuracy))
    print("-------------------------------------------------")
    print("( Here 1's  are positive 0's are negative)")
    len(train)

Results On Training Data
Using Gini Coefficient as impurity measure
-------------------------------------------------
True Positive are    2086
True Negatives are   6799
False Positive are   37
False Negatives are  67
Precsion is          0.9825718323127649
Recall is            0.9688806316767301
F1_Score is          4.099712368168744
Accuracy is          0.9884303037045278
-------------------------------------------------
( Here 1's  are positive 0's are negative)


8989

In [7]:
    GivenTestData =  val.copy()
    GivenTestData['predict']= [0]*len(GivenTestData)
    for i in list(GivenTestData.index.values):        
        GivenTestData.at[i,'predict'] = RecursiveFunctionToPredict(i,RootNode,GivenTestData)
    
    TN = 0
    FP = 0
    FN = 0
    TP = 0
    for i in list(GivenTestData.index.values):
        if str(GivenTestData.at[i,'left']) == str(1) and str(GivenTestData.at[i,'predict']) == str(1):
            TP = TP + 1
        elif str(GivenTestData.at[i,'left']) == str(1) and str(GivenTestData.at[i,'predict']) == str(0):
            FN = FN + 1
        elif str(GivenTestData.at[i,'left']) == str(0) and str(GivenTestData.at[i,'predict']) == str(1):
            FP = FP + 1
        elif str(GivenTestData.at[i,'left']) == str(0) and str(GivenTestData.at[i,'predict']) == str(0):
            TN = TN + 1
    print("Results On Validation Data")
    print("Using Gini Coefficient as impurity measure")
    print("-------------------------------------------------")
    print("True Positive are    "+ str(TP))
    print("True Negatives are   "+ str(TN))
    print("False Positive are   "+ str(FP))
    print("False Negatives are  "+str(FN))
    
    Precision = (TP/(TP+FP))
    Recall = (TP/(TP+FN))
    F1_Score = 2*((1/Recall)+(1/Precision))
    accuracy = (TN+TP)/(TN+TP+FP+FN)
    print("Precsion is          "+str(Precision))
    print("Recall is            "+str(Recall))
    print("F1_Score is          "+str(F1_Score))
    print("Accuracy is          "+str(accuracy))
    print("-------------------------------------------------")
    print("( Here 1's  are positive 0's are negative)")
    len(val)

Results On Validation Data
Using Gini Coefficient as impurity measure
-------------------------------------------------
True Positive are    495
True Negatives are   1699
False Positive are   28
False Negatives are  27
Precsion is          0.9464627151051626
Recall is            0.9482758620689655
F1_Score is          4.222222222222221
Accuracy is          0.9755446865273455
-------------------------------------------------
( Here 1's  are positive 0's are negative)


2249